# Lab 1: The NV centre in diamond – Spin Initialisation and readout
#### Setup A

Press Shift+Enter to run each code segment. The raw data generated by this notebook can be found in the folder './data/{todays_date}/'.


This lab uses the open-source qcodes library to manage experimental data. To work on this lab at home, you will need to install Python >3.7 and qcodes. 

Qcodes can be installed by running the command:

or in anaconda:

For futher information, visit: https://qcodes.github.io/Qcodes/start/index.html

In [ ]:
%matplotlib widget
import qcodes as qc
import qcodes.utils.validators as vals
from qcodes.plots.qcmatplotlib import MatPlot
from qcodes.loops import Loop
from qcodes.data.data_set import load_data
from qcodes.actions import Task, Wait
from qcodes.measure import Measure
from qcodes.utils import magic


from ultolib import (anritsu, korad, spincore)
from ultolib.spincore import pulse
import qcodes.instrument_drivers.stanford_research.SR830 as stanford_research

from scipy.optimize import curve_fit
import numpy as np

## Task 1.1: The Lock-In Amplifier and PulseBlaster 

There are 4 channels on the PulseBlaster that can be programmed to output unique pulse sequences, 'çh0' of the PulseBlaster is physically connected to the lock-in amplifier reference port. We will use this channel generate a 200 Hz reference signal for the lock-in amplifier.

In the next code segment, we connect the computer to the instruments. Only run this code segment ONCE! Otherwise, to connect to the instruments again, the kernel (i.e. the python backend) must be restarted and any unsaved data will be lost.

In [ ]:
pulse_blaster = spincore.PulseBlasterESRPRO(name='pulse_blaster', board_number=0)
pulse_blaster.core_clock(300)                     #Sets the clock speed, 
                                                  #must be called immediately after connecting to the PulseBlaster
lock_in_amp = stanford_research.SR830(name='lock_in_amp', address='GPIB0::13::INSTR')
microwave_src=anritsu.MG3681A('microwave_src', 'GPIB0::3::INSTR')
microwave_src.output('OFF')
microwave_src.output_level_unit('dBm')

station=qc.Station(pulse_blaster, lock_in_amp, microwave_src)
loc_provider = qc.data.location.FormatLocation(fmt='data/{date}/#{counter}_{name}_{time}')
qc.data.data_set.DataSet.location_provider = loc_provider

All channels of the PulseBlaster must be programmed simulatenously. Thus, to program the PulseBlaster, we must first define a pulse sequence for each channel and load it into a buffer and then program all the channels at once.

To define a pulse sequence we use a pulse object as shown below. In the 'PulseBlasterESRPRO' class, each channel (named 'chX' : x in {0,1,2,3}) is defined with a buffer. Prior to loading the buffer, we must empty it, so that the previous pulse sequence does affect the one we wish to program. We can do this by calling the 'reset_channel_buffer' method of the 'PulseBlasterESRPRO' class. We may then construct the pulse sequence. Simply put, a pulse sequence is composed of an array of 'pulse' objects which have a level (either 0 or 1 - corresponging to 0V or 3.3V at the PulseBlaster output) and a duration. Once this array is defined it can be loaded into the pulse sequence buffer of the corresponding channel via the channel's 'pulse_sequence_buffer.set()' method.

In [ ]:
ref_f = 200                           #The lock-in amplifier reference frequence.
ref_D = 0.5                           #The lock-in amplifier reference duty cycle.
T_ref_on = ref_D * 1 / ref_f
T_ref_off = (1 - ref_D) * 1 / ref_f

pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.

To program the PulseBlaster, and physically output the pulse sequence, the following code segment must be called.

In [ ]:
pulse_blaster.flush_channel_buffer()

To check that the lock-in amplifier is receiving a reference signal look to the reference display on the lock in amplifier (far right).  You should now see the reference frequency displayed. Change the reference frequency to 500 Hz, 1 kHz and 10 kHz and verify that the lock-in amplifier is receiving the reference signal.

In [ ]:
lock_in_amp.frequency()

Finally, remember to set the filter slope to 24 dB and set the reference frequency to 200 Hz by re-running the previous code segments.

In [ ]:
lock_in_amp.filter_slope(24)

## Task 1.2: Laser Modulation

On the lock-in amplifier front panel - 'Channel 1' - displays the DC value corresponding to the amplitude of the input signal at the reference frequency of the lock-in (which should be very close to 200 Hz at the moment).

Now we wish to measure something meaningful, copy the pulse sequence from 'ch0' to 'ch1'. This will turn the green laser on and off at the same frequency as the lock-in amplifier reference. The red photoluminescence collected from the NV sample will thus also be modulated at the same frequency. The photodiode will collect the modulated photoluminescence from the NV sample, and feed its voltage output to the lock-in amplifier.

In [ ]:
pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 0.
pulse_blaster.ch1.pulse_sequence_buffer.set([pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
    #TODO: Copy the pulse sequence from ch0 to ch1.
)                                     #Define the new pulse sequence for channel 1.
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.
pulse_blaster.flush_channel_buffer()

Set the lock-in amplifier time constant and sensivity using the code segment below.

In [ ]:
lock_in_amp.time_constant(100e-3)
lock_in_amp.sensitivity(50e-3)

Verify that you get a non-zero DC value on the lock-in amplifier, either by checking the front panel or running the bolow code segment. Ask your lab demonstrator to open the optics setup and have you view the emission of the sample (must wear goggles).

### Q1. Explain why you get a non-zero DC value on the lock-in amplifier output?

In [ ]:
lock_in_amp.R()

We now wish to measure the NV photoluminescence (PL) signal over time. To do so we must run a measurement. Measurements in this course consist of a loop, in which experimental parameter is varied and another parameter is measured. For optimising the (PL) signal, our dependent variable is 'time', and the dependant variable is the lock-in amplifier voltage, through which we measure the NV photoluminescence. 

To script this in python, we use a package called qcodes ('qc'), which allows us to simply define a measurement and the measurement parameters. We can define our indepedent variable, 'time', as a parameter (more specifically, a 'ManualParameter') as shown below. This tells qcodes that this is a parameter in our measurement that needs to be recorded. Additionally instruments have their own paramters, for example, the lock-in amplifier voltage 'R' is also qcodes parameter.

In [ ]:
time = qc.ManualParameter('time', unit='3 * time constant')

Now we need to tell python to:

1. Loop through a set of times.
2. Wait for the lock-in amplfier to settle.
3. Measure the the lock-in amplifier voltage.

To do that we use the 'Loop' class. The first argument - to instantiate the class - is the variable we wish to sweep through. For each settable parameter we can define the sweep values by calling its sweep method. For example, 'time.sweep(start=0, end=10, step=1)' tells python to sweep 'time' from 0 to 10 in steps of 1. The next argument is the delay which we use to tell python to wait 3 * the lock-in amplifier time constant. This allow lock-in amplifier voltage to settle. Finally to tell python to measure the lock-in amplifier voltage ('R') for each value of 'time', we call the 'each' method of the loop, and pass the lock-in amplifier voltage parameter ('lock_in_amp.R') as an argument.

When the measurement is run, the data is automatically saved. To get the data to, for example, plot the measurement, we call the function 'loop.get_data_set'. Note that the dataset in this measurement is the variable 'data_tuning'.

In [ ]:
loop = Loop(time.sweep(0, 10, step=1), 
            delay = 3 * lock_in_amp.time_constant()).each(lock_in_amp.R)
data_tuning = loop.get_data_set(name='tuning')
#Plot the measurement
plot = MatPlot(data_tuning.lock_in_amp_R)
loop.with_bg_task(plot.update)

In [ ]:
#Run the measurement
loop.run()
plot.update()

## Task 1.3: Laser Modulation

200 Hz corresponds to a laser pulse length of 2.5 ms. This is a relatively long pulse duration forthe NV centers. We only require the laser pulse length to be 1μs to 10μs long for polarising the NV spins to the |0〉state. 

Keeping the frequency of ch0 (lock-in modulation) same as Task 1.2, change the frequency of Ch1 to 100 kHz pulsing continuously. This is actually trickier than it seems. The PulseBlaster has the property that it will only repeat the total pulse sequence once all of the pulse sequences of the idividual channels have completed. Thus, if we define a pulse sequence such as:

In [ ]:
laser_f = 100e3                              #Laser modulation frequency.
laser_D = 0.5                                #Laser modulation duty cycle.
T_laser_on = laser_D * 1 / laser_f            
T_laser_off = (1 - laser_D) * 1 / laser_f
N_laser_pulses = round(laser_f / ref_f)      #Number of laser pulses that can fit in the reference period.

pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 0.
pulse_blaster.ch1.pulse_sequence_buffer.set(
    [[pulse(level=1, duration=T_laser_on), pulse(level=0, duration=T_laser_off)]]
)                                     #Define the new pulse sequence for channel 1.
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.

We can see that there will be only one 5 us laser pulse in one period of the reference pulse. In other words the laser modulation frequency is equal to the reference frequency and the laser modulation duty cycle is actually 0.5 / (100e3 * 200) = 2.5e-8. Therefore, we must calculate the number of laser pulses ('N_laser_pulses') that fit within one period of the lock-in reference. The laser modulation pulse sequence is then [pulse(level=1, duration=5e-6), pulse(level=1, duration=5e-6) ... repeated N_laser_pulses], as shown below.

In [ ]:
laser_f = 100e3                              #Laser modulation frequency.
laser_D = 0.5                                #Laser modulation duty cycle.
T_laser_on = laser_D * 1 / laser_f           #Laser on time. 
T_laser_off = (1 - laser_D) * 1 / laser_f    #Laser off time.
N_laser_pulses = round(laser_f / ref_f)      #Number of laser pulses that can fit in the reference period.

pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 0.
pulse_blaster.ch1.pulse_sequence_buffer.set(
    [[pulse(level=1, duration=T_laser_on), pulse(level=0, duration=T_laser_off)] for i in range(0, N_laser_pulses)]
)                                     #Define the new pulse sequence for channel 1.
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.
pulse_blaster.flush_channel_buffer()

Note how we can construct large looped pulse sequence arrays by nesting smaller pulse sequence arrays. 

Make sure the DC value goes to 0. By running the following code segment or looking at the front panel of the lock-in amplifier.

In [ ]:
lock_in_amp.R()

### Q2. Explain why the DC value goes to 0?

## Task 1.4: Longitudinal $T_1$ Relaxation Time

The laser pulse sequence for this task will need to be altered. The new pulse sequence will consist of pulses in the following order:
1. Initialisation pulse of 500μs length. 
2. A first Readout pulse of 500μs length after a time delay τ.
3. A second Readout pulse of same length that is 180◦ away from the first readout pulse.

The delay τ can then be swept from 100μs to 10 ms to obtain a T1 signature exponential decay signal. Ideally we would only require one initialisation and one readout pulse. However, since our laser is also effectively modulated at the same frequency as the envelope, it would drown out the weak T1 signal. Think about what signal component the lock-in apmplifier measures from the spin signal and from the photoluminescence signal. This experiment requires a very low reference frequency ('ref_f' = 21 Hz). This is because the pulses separated by the delay τ must be able to fit within the envelope period, and the idle time after the pulses needs to be much longer than T1.

In [ ]:
ref_f = 21
ref_D = 0.5
T_ref_on = ref_D * 1 / ref_f
T_ref_off = (1 - ref_D) * 1 / ref_f

T_laser_on = 5e-4
T_laser_off = T_ref_off - T_laser_on

tau = qc.ManualParameter('tau', initial_value=200e-6, unit='s')
def get_T_idle():
    return T_ref_on - 2 * T_laser_on - tau()
T_idle = qc.Parameter('T_idle', get_cmd=get_T_idle, unit='s')

pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 0.
pulse_blaster.ch1.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_laser_on), pulse(level=0, duration=tau), 
     pulse(level=1, duration=T_laser_on), pulse(level=0, duration=T_idle),
     pulse(level=1, duration=T_laser_on), pulse(level=0, duration=T_laser_off)]
)                                     #Define the new pulse sequence for channel 1.
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.
pulse_blaster.flush_channel_buffer()

Run the experiment. 

In [ ]:
lock_in_amp.time_constant(1)
lock_in_amp.sensitivity(100e-6)

loop = Loop(tau.sweep(200e-6, 10e-3, step=200e-6)).each(
    Task(pulse_blaster.flush_channel_buffer),
    Wait(10 * lock_in_amp.time_constant()),
    lock_in_amp.R
)
#Plot the measurement
data_T1 = loop.get_data_set(name='T1')
plot = MatPlot(data_T1.lock_in_amp_R)
plot.tight_layout()
loop.with_bg_task(plot.update)

In [ ]:
#Run the measurement
loop.run()
plot.update()

To extract the $T_1$ time the decay must be fit. Below we define the fit model as a decaying exponential of the form $ae^{-\frac{\tau}{T_1}} + c$. Here, $a$ defines the magnitude of the spin signal at $\tau = 0$. i.e. the spin is fully polarised in the $|0\rangle$ state and $c$ is the offset photoluminescence. We define the fit model as a python function, with first argument $\tau$ (labeled as 'x' in the code) and the next three arguments are $a, T_1, c$. Our aim is to find the $\{a, T_1, c\}$ which produces a curve which most closely matches the experimental data. As this is non-linear fit, we must also provide the initial guess for the fit parameters $\{a, T_1, c\}$. This is an array of the $[{a}, {T_1}, {c}]$. From the data you have measured, choose the appropriate start points.

In [ ]:
                    #['a', 'T_1',   'c' ]   
initial_fit_params = [4e-6, 3e-3, 49e-6]
def fit_model(x, a, b, c):
    return a * np.exp(-1.0 * x / b) + c

At some point you will need to directly access the experimental data you have produced. Following the experiment, the data is stored in a DataArray stucture, where it is not directly accessible. The below code segment extracts the data from the DataArray structure into a standard numpy array. The format for this is {data_set_name}.{parameter_name}.ravel(). In the below example, we wish to extract 'tau' and the lock-in amplifier magnitude 'R'. In the dataset, these are stored as 'data_T1.tau_set' and 'data_T1.lock_in_amp_R'. Note, the addition of the suffix '_set' for 'tau'. This refers to the fact that we had swept 'tau'. 

For additional information on how to access the datasets belonging to previous experiments see the Appendix A.

In [ ]:
xdata = data_T1.tau_set.ravel()
ydata = data_T1.lock_in_amp_R.ravel()

The following code segment performs the actual fit. 'fit_params' is an array of the form $[a, T_1, c]$ with the optimum values of $a, T_1$ and $c$. 'fit_cov' is the covariance matrix. The error (1-sigma) in the estimate of the fit parameters is the diagonal of this matrix. i.e. the error in $a$ is the $\sqrt{fit\_cov[0][0]}$ and the error in $T_1$ is $\sqrt{fit\_cov[1][1]}$ and so forth.

In [ ]:
fit_params, fit_cov = curve_fit(fit_model, xdata, ydata, initial_fit_params)

To ensure the data is saved similarly to the experiment data we run a dummy measurement with parameters: 'fit_T1' and 'fit_T1_error'. Now it is possible to access the $T_1$ and its error in the same way you would access the experimental data.

In [ ]:
fit_T1 = qc.ManualParameter(name='fit_T1', unit='s', initial_value=fit_params[1])
fit_T1_err = qc.ManualParameter(name='fit_T1_err', unit='s', initial_value=np.sqrt(fit_cov[1][1]))
meas_T1_fit = Measure(fit_T1, fit_T1_err)
data_T1_fit_params = meas_T1_fit.get_data_set(name='T1_fit_params')
meas_T1_fit.run()

Similarly we run a dummy experiment where we 'measure' (i.e. call the fit model function for every values in the fit axis) the fit curve and save the data in a new dataset labelled 'data_T1_fit'. Here the fit axis is the parameter 'tau_fit' and the fit curve is the parameter 'R_fit'.

In [ ]:
R_fit = qc.ManualParameter(name='R_fit', unit='V')
def set_fit_curve(ax_val):
    R_fit.set(fit_model(ax_val, *fit_params))   
tau_fit = qc.Parameter(name='tau_fit', label='tau', 
                        unit='s', set_cmd=set_fit_curve)

loop = Loop(tau_fit.sweep(xdata.min(), xdata.max(), num=100)).each(R_fit)
data_T1_fit = loop.get_data_set(name='T1_fit')
loop.run()
plot = MatPlot([data_T1.lock_in_amp_R, data_T1_fit.R_fit])

The $T_1$ is:

In [ ]:
fit_T1()

## Task 1.5: Optically Detected Magnetic Resonance

So far, we have used Channel 0 for reference signal and Channel 1 for the laser pulse sequence.  Now we would like to introduce microwave radiation to the sample.  The pulse sequence for this will be defined in Channel 2.

Set ch0 back to 200 Hz, and ch1 at 100 kHz pulsing continuously. For the microwave, we would like to create something similar to an inverted copy of the laser pulse sequence. Furthermore, Channel 2 must be modulated such that it follows the envelope of the reference signal in Channel 0. If you imagine that the reference (R), laser (L) and microwave (M) as digital signals, then M = R & (~L).

Your pulse sequence should look similar to Figure 4. Create this sequence and use the pulse sequence plotting code so you can really see what the PulseBlaster is outputting.

### Q3. Explain why the microwave pulses must be modulated to follow the envelope of the reference signal?

In [ ]:
ref_f = 200                                  #Reference frequency.
ref_D = 0.5                                  #Reference duty cycle.
T_ref_on = ref_D * 1 / ref_f                 #Reference time on.
T_ref_off = (1 - ref_D) * 1 / ref_f          #Reference time off.

#TODO: Specifiy these constants.
laser_f =                                    #Laser modulation frequency.
laser_D =                                    #Laser modulation duty cycle.
T_laser_on =                                 #Laser on time. 
T_laser_off =                                #Laser off time.
N_laser_pulses =                             #Number of laser pulses that can fit in the reference period.

mw_f = laser_f                               #Microwave modulation frequency.
mw_D = laser_D                               #Microwave modulation duty cycle.
T_mw_on = mw_D * 1 / mw_f                    #Microwave time on.
T_mw_off = (1 - mw_D) * 1 / mw_f             #Microwave time off.
N_mw_pulses = int(ref_D * N_laser_pulses)    #Number of microwave pulses that can fit in the reference period.

pulse_blaster.reset_channel_buffer()  #Clear the previous pulse sequence.
pulse_blaster.ch0.pulse_sequence_buffer.set(
    [pulse(level=1, duration=T_ref_on), pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 0.
pulse_blaster.ch1.pulse_sequence_buffer.set(
    #TODO: Enter the laser pulse sequence.
)                                     #Define the new pulse sequence for channel 1.
pulse_blaster.ch2.pulse_sequence_buffer.set(
    [[[pulse(level=0, duration=T_mw_off), pulse(level=1, duration=T_mw_on)] for i in range(0, N_mw_pulses)],
     pulse(level=0, duration=T_ref_off)]
)                                     #Define the new pulse sequence for channel 2.
pulse_blaster.plot_channel_buffer()   #This function plots the newly defined pulse sequence.
pulse_blaster.flush_channel_buffer()

Turn on the microwave and set the lock-in amplifier to the correct settings. And then measure the lock-in amplifier output

In [ ]:
lock_in_amp.time_constant(100e-3)
lock_in_amp.sensitivity(50e-6)

microwave_src.power(11)
microwave_src.pulse_modulation('EXT')

microwave_src.frequency(2.8e9)
microwave_src.output('ON')

In [ ]:
lock_in_amp.R()

### Q4. At this point the lock-in amplifier output is still ~0 V. Explain why this is the case?

### Q5. What do you need to do to see a non-zero signal?

In [ ]:
loop = Loop(microwave_src.frequency.sweep(2.8e9, 2.95e9, num=100),
            delay=5*lock_in_amp.time_constant()).each(lock_in_amp.R)
#Plot the measurement
data_ODMR = loop.get_data_set(name='ODMR')
plot = MatPlot(data_ODMR.lock_in_amp_R)
plot.tight_layout()
loop.with_bg_task(plot.update)

In [ ]:
#Run the measurement
loop.run()
plot.update()

### Q6. Identify, describe and note down all the salient features you observe in your magnetic resonance sweep. Can you guess if there is an external magnetic field applied near your diamond sample?

## Turn off the Instruments

Please run this code segment after you have completed the lab.

In [ ]:
pulse_blaster.stop()
microwave_src.output('OFF')

## Appendix A: Accessing Your Data

You may run a measurement many times - let's say 5 times - and decide that the third run is the measurement you would like to plot for your lab report. Or you would like to access the data to perform some analysis. Unfortunately, the notebook is showing the latest run, run 5. Thus, you must be able to import your data into the notebook. This can be done by calling the function 'load_data' the data location as the argument. The data location for each experiment, is saved in a directory formatted as ./data/YYYY-MM-DD/#{Three-digit experiment id}_{Experiement name}_hh_mm_ss'. This location is also figure title for any experiments run. For example:

In [ ]:
test_data1 = load_data('./data/2021-09-20/#001_test_data1_13-13-07')
test_data2 = load_data('./data/2021-09-20/#002_test_data2_13-13-50')

You can then plot the data by calling the function 'MatPlot' as shown below. When using 'Matplot', multiple dataset may be plotted. If supplied in the argument list as Matplot(dataset1, dataset2, ...), 'MatPlot' will create subplots with each dataset plotted within its own subplot. If the datasets are supplied as Matplot([dataset1, dataset2, ...]), the datasets will be plotted on the same axis.

In [ ]:
MatPlot(test_data1.y)
MatPlot(test_data1.y, test_data2.y)
MatPlot([test_data1.y, test_data2.y])

Convert the data into a numpy array for data processing or curve fitting can be done so by calling the 'reval' method of the parameter you would like to convert in the data set. Remember to add the '_set' suffix if the paramter was swept. 

In [ ]:
test_data1.x_set.ravel()
test_data1.y.ravel()